# Function calling with Gemini

This notebook provides a guide to using Google's Gemini API for function calling. We will explore how to enable AI models to interact with external functions, APIs, and tools.

Function calling (also known as tool calling) allows language models to interact with external functions, APIs, and tools. Instead of just generating text, the model can call predefined functions with appropriate parameters and make decisions about which functions to call based on context. It can receive function results and incorporate them into responses and continue the conversation with context from function outputs.

### How function calling works
1. **Function definition**: Define functions with clear schemas.
2. **Model decision**: The model decides when and which function to call.
3. **Parameter extraction**: The model extracts appropriate parameters from context.
4. **Function execution**: The function is called with extracted.parameters.
5. **Result integration**: Function results are integrated back into the conversation

Function calling is useful because it allows access to real-time data like weather or stocks, enables integration with APIs or databases, can trigger actions such as sending emails or booking appointments, and returns structured data instead of plain text.

In [1]:
import os
import json
import requests
from datetime import datetime
from typing import Dict, Any, List
from dotenv import load_dotenv
from google import genai
from google.genai import types
from google import generativeai

# Load environment variables
load_dotenv()

# Set up API key
generativeai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

- `typing` imports provide type hints for better code clarity and IDE support.
- `google.generativeai` is the official Google SDK for Gemini
- `genai.configure()` sets up the API key for all subsequent requests

In [2]:
# List available models
print("Available Gemini models:")
for model in generativeai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(f"- {model.name}")

Available Gemini models:
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-002
- models/gemini-1.5-pro
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash
- models/gemini-1.5-flash-002
- models/gemini-1.5-flash-8b
- models/gemini-1.5-flash-8b-001
- models/gemini-1.5-flash-8b-latest
- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-05-20
- models/gemini-2.5-flash
- models/gemini-2.5-flash-lite-preview-06-17
- models/gemini-2.5-pro-preview-05-06
- models/gemini-2.5-pro-preview-06-05
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-fla

- `genai.list_models()` returns all available models

We filter for models that support `generateContent` (text generation). This helps identify which models support function calling

### Basic function calling

#### Define a mock function

Let’s simulate a weather API function. This mock implementation returns fixed values and serves as a placeholder during development or testing. Mock implementation means that it doesn't connect to a real weather API. It simulates behavior using hardcoded values (Mock functions are useful during prototyping, testing, or tutorials where external API setup is unnecessary or unavailable).

In [3]:
def get_current_weather(location: str, unit: str = "celsius") -> Dict[str, Any]:
    """
    Get current weather information for a specific location.

    Args:
        location: The city and state/country (e.g., "New York, NY")
        unit: Temperature unit ("celsius" or "fahrenheit")

    Returns:
        Dictionary containing weather information
    """
    # Simulate weather API call (in real implementation, we will use actual weather API)
    weather_data = {
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "description": "Partly cloudy",
        "humidity": 65,
        "wind_speed": 10
    }

    print(f"🌤️  Getting weather for {location}...")
    return weather_data


# Test the function
test_result = get_current_weather("London, UK")
print(f"Test result: {test_result}")

🌤️  Getting weather for London, UK...
Test result: {'location': 'London, UK', 'temperature': 22, 'unit': 'celsius', 'description': 'Partly cloudy', 'humidity': 65, 'wind_speed': 10}


This function acts as a standalone unit of business logic that Gemini will eventually be able to call.
- The function includes proper type hints for parameters and return values.
  - Input: `location` as `str`, `unit` as optional `str` (default "celsius").
  - Output: `Dict[str, Any]` for flexible, JSON-like structured return data.
- The triple-quoted string immediately under the function is called a docstring. Docstring provides clear description of purpose, parameters, and return format.
- Instead of making an actual API call, it uses hardcoded values: temperature varies depending on unit, Weather data like humidity, description, and wind speed are fixed.

The function simulates an API call (in production, we would call a real weather service). This lays the groundwork for enabling the model to interact with external logic, such as APIs or backend services.

#### Creating function schema
To enable Gemini to understand and call a function, we need to formally describe that function in a way the model can interpret. This is done using a JSON schema, which defines what the function is called, what it does, and what inputs it expects — including their types, required fields, and constraints.

We can think of the schema like a contract between our function and the model. Gemini uses this schema to decide whether the function is relevant to a given user query, what arguments should be passed into the function, and whether it has enough information to perform a call.

The schema does not include the actual implementation of the function — only its signature and input specification. This is what we register with Gemini's tool interface so that the model can intelligently match user prompts to external tool logic.

In [4]:
# Define the function declaration using proper JSON Schema
weather_function = {
    "name": "get_current_weather",
    "description": "Fetches the full current weather report including temperature, humidity, wind speed, and conditions.",  # Helps the model understand when this tool should be used
    "parameters": {
        "type": "object",  # The input to the function is expected to be a single object (i.e., a dictionary of named parameters)
        "properties": {  # This defines the structure and validation for each expected parameter
            "location": {
                "type": "string",
                "description": "City and country (e.g., 'New York, NY')"  # Human-readable guidance for the model
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],  # Only allow two possible values for temperature unit
                "description": "Temperature unit"
            }
        },
        "required": ["location"]  # The 'location' field must be provided by the model for the function to be called
    }
}

* The `"name"` field must match the actual function name in our Python code (`get_current_weather`).
* The `"description"` is used by the model during inference. It helps Gemini know when to use this tool and explain it in internal decision-making.
* The `"parameters"` key defines the input structure the model needs to provide when calling the function. It uses the JSON Schema standard — a widely-used convention for describing and validating structured data.
  * Each entry under `"properties"` represents one expected input to the function.
  * `"location"` is a required string input — this will be something like `"London, UK"`.
  * `"unit"` is an optional string that can only be `"celsius"` or `"fahrenheit"` (thanks to the `"enum"` constraint).
  * The `enum` ensures the model won't make up invalid units like `"kelvin"` or `"degrees"`.
* The `"required"` list enforces that the function must be called with a valid `"location"` field. If the user prompt lacks this, Gemini will either skip the function or ask a follow-up to gather that missing piece.

This schema does not provide default values. If we wanted the model to assume defaults automatically, we could explicitly include `"default"` fields in the schema — but here it is kept minimal and clean.


- **Schema Format**: Uses JSON Schema specification for parameter validation
- **name**: Must match the actual function name exactly
- **description**: Helps the model understand when to use this function
- **parameters**: Defines the expected input structure
- **properties**: Specifies each parameter's type and description
- **required**: Lists mandatory parameters
- **enum**: Restricts values to specific options
- **default**: Provides fallback values for optional parameters

#### Basic function call example
Now that we have defined a schema for our function and implemented the logic, it's time to bring the whole pipeline together: letting Gemini recognize when to call a tool and extract arguments from natural language input.

Function calling adds reasoning and interactivity to static prompts. Instead of hardcoding logic into prompts, we allow Gemini to choose when to use tools — making your application smarter, modular, and easier to maintain.


In [5]:
  # Register the function schema as a Gemini tool
  client = genai.Client()  # Initialize the Gemini API client
  # Wrap the schema in a Tool object – this tells Gemini what functions it can potentially call
  weather_tool = types.Tool(function_declarations=[weather_function])
  # Configuration for content generation – includes registered tools
  config = types.GenerateContentConfig(tools=[weather_tool])

  # Prompt
  prompt = "What's the weather like in Tokyo, Japan?"

  # Step 1: Send the request to Gemini, including the prompt and tool configuration
  response = client.models.generate_content(
      model="gemini-2.5-flash",
      contents=prompt,
      config=config,  # This allows Gemini to consider available tools
  )

  # Step 2: Extract the model's decision (whether it wants to call a function/tool)
  function_call = response.candidates[0].content.parts[0].function_call  # This is where Gemini tells us if it wants to call a tool

  # Step 3: If the model decided to call a tool
  if function_call:
      print(f"🔧 Function to call: {function_call.name}")
      print(f"📋 Arguments: {function_call.args}")

      # Step 4: Execute the function locally with extracted arguments
      if function_call.name == "get_current_weather":
          result = get_current_weather(**dict(function_call.args))
          print("✅ Function result:", result)

          # Send the function result back to the model to get natural language response
          # Step 5: Create a new conversation with the function result
          conversation = [
              {"role": "user", "parts": [{"text": prompt}]},
              {"role": "model", "parts": [{"function_call": function_call}]},
              {"role": "user", "parts": [{"function_response": {
                  "name": function_call.name,
                  "response": result
              }}]}
          ]

          # Step 6: Get the final natural language response
          final_response = client.models.generate_content(
              model="gemini-2.5-flash",
              contents=conversation,
              config=config,
          )

          print("💬 Final natural language response:")
          for part in final_response.candidates[0].content.parts:
              if hasattr(part, 'text') and part.text:
                  print(f"   Text: {part.text}")
              elif hasattr(part, 'thought_signature'):
                  print(f"   Thought: {part.thought_signature}")
              else:
                  print(f"   Other part type: {type(part)}")
  # If the model responded with plain text instead of a function call
  else:
      print("💬 No function call. Model responded:", response.text)

🔧 Function to call: get_current_weather
📋 Arguments: {'location': 'Tokyo, Japan'}
🌤️  Getting weather for Tokyo, Japan...
✅ Function result: {'location': 'Tokyo, Japan', 'temperature': 22, 'unit': 'celsius', 'description': 'Partly cloudy', 'humidity': 65, 'wind_speed': 10}
💬 Final natural language response:
   Text: The weather in Tokyo, Japan is partly cloudy with a temperature of 22 degrees Celsius, 65% humidity, and a wind speed of 10 km/h.


- The function schema (`weather_function`) is registered as a tool using the `Tool` wrapper. This acts as a contract the Gemini model uses to recognize when and how to call this function. The model won’t use any external logic unless we explicitly register it this way.
- The `GenerateContentConfig` binds the tools to the generation context. This ensures the model is aware of available tools at the time of content generation.
- The prompt is a regular natural language question about the weather. The model processes this and decides whether to respond with text or invoke a function. Behind the scenes, Gemini analyzes the tools available and tries to match them to the user intent.
- If the function name matches our implementation, we call it locally using Python’s unpacking (`**dict(...)`). This simulates what would happen in a real system, where backend logic would be triggered based on Gemini’s output.
- After executing the function, the result (a dictionary) is formatted into a `function_response` block. This is sent back to Gemini as part of a new `conversation` payload, mimicking how it would see the result if calling an external system.

This pattern — prompt → tool call → local execution → structured result → natural language summary — is the essence of tool-augmented generation in Gemini. It allows the model to go beyond static language generation and become an intelligent agent connected to real-world logic and data.

### Function calling integration with ongoing conversations
In real-world applications, users interact with AI assistants through ongoing conversations rather than isolated prompts. Function calling in Gemini is designed to work within these multi-turn exchanges, preserving context and memory as the chat evolves.

Instead of rebuilding the conversation manually each time a function is triggered, a more scalable approach is to maintain a single, persistent conversation history. Every new user message and model response—whether plain text, function call, or function response—is appended to this history. This allows Gemini to reason over the entire interaction timeline and make decisions based on full conversational context.

In [6]:
# Initialize Gemini client and tool configuration
client = genai.Client()

# Register the function schema once
weather_tool = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[weather_tool])

# Persistent conversation history
conversation_history = []  # Keeps track of all chat turns

def send_and_handle_message(message_text: str):
    # Append the latest user message to history
    conversation_history.append({"role": "user", "parts": [{"text": message_text}]})

    # Request model output with full chat history
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=conversation_history,
        config=config,
    )

    candidate = response.candidates[0].content
    function_call = candidate.parts[0].function_call if candidate.parts and hasattr(candidate.parts[0], 'function_call') else None

    if function_call:
        print(f"🔧 Function to call: {function_call.name}")
        print(f"📋 Arguments: {function_call.args}")

        if function_call.name == "get_current_weather":
            result = get_current_weather(**dict(function_call.args))
            print("✅ Function result:", result)

            # Add model's function call and user's function response to history
            conversation_history.append({"role": "model", "parts": [{"function_call": function_call}]})
            conversation_history.append({"role": "user", "parts": [{"function_response": {
                "name": function_call.name,
                "response": result
            }}]})

            # Ask Gemini to generate final reply considering everything so far
            final_response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=conversation_history,
                config=config,
            )

            print("💬 Final natural language response:")
            for part in final_response.candidates[0].content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"   Text: {part.text}")

            # Add final model response to history
            conversation_history.append({"role": "model", "parts": final_response.candidates[0].content.parts})

    else:
        # If no function call, handle as a normal response
        print("💬 Model response (no function call):")
        for part in candidate.parts:
            if hasattr(part, 'text') and part.text:
                print(f"   Text: {part.text}")

        conversation_history.append({"role": "model", "parts": candidate.parts})

# Example usage
send_and_handle_message("Hi!")
send_and_handle_message("What's the weather like in Tel Aviv, Israel?")
send_and_handle_message("Thanks, and do you know what's the weather like in Mumbai, India?")

💬 Model response (no function call):
   Text: Hello! How can I help you today?

🔧 Function to call: get_current_weather
📋 Arguments: {'location': 'Tel Aviv, Israel'}
🌤️  Getting weather for Tel Aviv, Israel...
✅ Function result: {'location': 'Tel Aviv, Israel', 'temperature': 22, 'unit': 'celsius', 'description': 'Partly cloudy', 'humidity': 65, 'wind_speed': 10}
💬 Final natural language response:
   Text: The weather in Tel Aviv, Israel is partly cloudy with a temperature of 22 degrees Celsius, 65% humidity, and a wind speed of 10.
🔧 Function to call: get_current_weather
📋 Arguments: {'location': 'Mumbai, India'}
🌤️  Getting weather for Mumbai, India...
✅ Function result: {'location': 'Mumbai, India', 'temperature': 22, 'unit': 'celsius', 'description': 'Partly cloudy', 'humidity': 65, 'wind_speed': 10}
💬 Final natural language response:
   Text: The weather in Mumbai, India is partly cloudy with a temperature of 22 degrees Celsius, 65% humidity, and a wind speed of 10.


- One persistent `conversation_history` list is used throughout multiple function calls and prompts.
- All roles (`user`, `model`) and parts (`text`, `function_call`, `function_response`) are appended to this list as the conversation evolves.

### Parallel function calling

In many real-world scenarios, users ask questions that require multiple pieces of information simultaneously. For example, "What's the weather in New York and London?" or "Get me the weather for Tokyo and also search for restaurants nearby." Instead of making sequential function calls, Gemini supports parallel function calling, which allows the model to invoke multiple functions at once, improving efficiency and user experience.

Parallel function calling is particularly useful when:
- Multiple independent data sources need to be queried
- Functions don't depend on each other's results
- We want to reduce response time by avoiding sequential execution
- The user's query naturally requires multiple pieces of information

#### Adding more functions for parallel calling
Let's expand our toolkit by adding a restaurant search function alongside the existing weather function to demonstrate parallel calling with multiple different functions. As before, this is a simulated (mock) implementation, focusing on the interaction structure rather than using a real-world API. The goals here is to demonstrate how we can expand Gemini’s toolset with multiple functions.

In [7]:
def search_restaurants(location: str, cuisine: str = "any", price_range: str = "moderate") -> Dict[str, Any]:
    """
    Search for restaurants in a specific location.

    Args:
        location: The city and state/country (e.g., "New York, NY")
        cuisine: Type of cuisine (e.g., "italian", "japanese", "mexican")
        price_range: Price range ("budget", "moderate", "expensive")

    Returns:
        Dictionary containing restaurant information
    """
    # Mock restaurant data (in real implementation, use actual restaurant API)
    restaurants = {
        "location": location,
        "cuisine": cuisine,
        "price_range": price_range,
        "results": [
            {
                "name": "Bella Vista",
                "rating": 4.5,
                "price": "$$",
                "cuisine": cuisine if cuisine != "any" else "Italian"
            },
            {
                "name": "Golden Dragon",
                "rating": 4.2,
                "price": "$$$",
                "cuisine": cuisine if cuisine != "any" else "Chinese"
            },
            {
                "name": "Local Bistro",
                "rating": 4.0,
                "price": "$",
                "cuisine": cuisine if cuisine != "any" else "American"
            }
        ]
    }
    print(f"🍽️  Searching restaurants in {location}...")
    # Return structured data back to Gemini for integration into its reply
    return restaurants

# Test the function
test_restaurants = search_restaurants("Paris, France", "french")
print(f"Test result: {test_restaurants}")

🍽️  Searching restaurants in Paris, France...
Test result: {'location': 'Paris, France', 'cuisine': 'french', 'price_range': 'moderate', 'results': [{'name': 'Bella Vista', 'rating': 4.5, 'price': '$$', 'cuisine': 'french'}, {'name': 'Golden Dragon', 'rating': 4.2, 'price': '$$$', 'cuisine': 'french'}, {'name': 'Local Bistro', 'rating': 4.0, 'price': '$', 'cuisine': 'french'}]}


* `search_restaurants` is defined with three input parameters: `location` (required), and `cuisine` and `price_range` (both optional with default values). This structure mimics real-world function design for user customization.
* Inside the function, a static dictionary simulates restaurant search results, returning structured information about three restaurant options. The cuisine type is dynamically adjusted if the user specifies it, otherwise default examples are used.
* The function returns a Python dictionary formatted in a way Gemini can consume. This allows Gemini to turn structured function results into conversational text for the user.

This prepares the foundation for integrating parallel tool use with Gemini: both weather lookup and restaurant search can now be included in the `GenerateContentConfig` tools list. Gemini can decide whether to call one or both tools depending on user input, rather than requiring sequential steps or manual triggers.

#### Creating function schemas for parallel calling
When building conversational systems that handle multiple function calls dynamically, it is essential that the model understands not just how to call a function, but exactly what parameters it expects. This ensures Gemini has a contract it can follow when suggesting arguments, validating input, and producing structured output that developers can trust.

To achieve this, each function must be described using a function schema. These schemas specify the function name, a description of its purpose, the parameters it takes, and what types and constraints apply to those parameters. Now let's define the schema for our restaurant search function:

In [8]:
# Define the schema for the restaurant search function — tells Gemini how to call this tool correctly
restaurant_function = {
    "name": "search_restaurants",  # Internal function name Gemini uses to identify this tool
    "description": "Search for restaurants in a specific location with optional cuisine and price filters.",  # Plain language description for reasoning
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",  # User must provide this as a plain text string
                "description": "City and country (e.g., 'Paris, France')"
            },
            "cuisine": {
                "type": "string",
                "description": "Type of cuisine (e.g., 'italian', 'japanese', 'mexican')"  # Optional — adds flexibility
            },
            "price_range": {
                "type": "string",
                "enum": ["budget", "moderate", "expensive"],
                "description": "Price range preference"
            }
        },
        "required": ["location"]  # Only 'location' is mandatory for this function to work
    }
}

# Register both weather and restaurant tools so Gemini can choose from both in a single conversation
client = genai.Client()
multi_tool = types.Tool(function_declarations=[weather_function, restaurant_function])  # Combines both schemas into one toolset
config = types.GenerateContentConfig(tools=[multi_tool])  # Pass combined tools as part of generation configuration

* `restaurant_function` contains metadata about the `search_restaurants` function, including the name Gemini uses to recognize and call it, a description that guides the model’s reasoning about when to use it, and the `parameters` structure, which follows a JSON Schema–style definition. This helps Gemini validate inputs automatically before even suggesting a function call.
* Both `weather_function` (defined earlier) and `restaurant_function` are wrapped together in a `types.Tool` object. This tells Gemini it has two options available whenever content is generated.
* `GenerateContentConfig` is initialized with the `multi_tool`. This configuration is passed to any content generation request where we want Gemini to have access to both tools simultaneously.

These steps create a contract between our application logic and Gemini. If a parameter is left out, or a function isn’t registered, Gemini simply won’t use it — making this schema step a critical part of maintaining reliability and predictable behavior.

#### Parallel function calling example
Here's how to handle parallel function calls when Gemini decides to call multiple functions simultaneously:

In [9]:
def handle_parallel_function_calls(prompt: str):
    """
    Handle a prompt that might trigger multiple function calls simultaneously.
    This function sends the prompt to Gemini, processes all detected function calls, executes them, and then asks Gemini to generate a final response based on all results.
    """
    print(f"📝 User prompt: {prompt}")
    print("=" * 50)

    # Send the user prompt to Gemini with tool configuration enabled
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,  # Pass the raw user text as contents
        config=config,  # Use the configuration with both tools registered
    )

    candidate = response.candidates[0].content  # Extract the primary response candidate

    # Check if there are any function calls detected in the response parts
    function_calls = []
    for part in candidate.parts:
        if hasattr(part, 'function_call') and part.function_call:
            function_calls.append(part.function_call)

    if function_calls:
        print(f"🔧 Found {len(function_calls)} function call(s):")

        # Placeholder for collecting all results
        function_results = []

        # Execute the correct function based on function name
        for i, function_call in enumerate(function_calls):
            print(f"\n  {i+1}. Function: {function_call.name}")
            print(f"     Arguments: {dict(function_call.args)}")

            # Execute the correct function based on function name
            if function_call.name == "get_current_weather":
                result = get_current_weather(**dict(function_call.args))
            elif function_call.name == "search_restaurants":
                result = search_restaurants(**dict(function_call.args))
            else:
                result = {"error": f"Unknown function: {function_call.name}"}

            function_results.append((function_call, result))  # Store both call and result
            print(f"     Result: {result}")

        # Build conversation with all function calls and responses
        conversation = [
            {"role": "user", "parts": [{"text": prompt}]},
            {"role": "model", "parts": [{"function_call": fc} for fc, _ in function_results]},
        ]

        # Prepare all function responses as user messages
        function_response_parts = []
        for function_call, result in function_results:
            function_response_parts.append({
                "function_response": {
                    "name": function_call.name,
                    "response": result
                }
            })

        conversation.append({"role": "user", "parts": function_response_parts})

        # Ask Gemini to generate a final human-readable message based on all previous steps
        final_response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=conversation,
            config=config,
        )

        print("\n💬 Final natural language response:")
        for part in final_response.candidates[0].content.parts:
            if hasattr(part, 'text') and part.text:
                print(f"   {part.text}")

    else:
        # Handle the case where Gemini produces only plain text with no function call
        print("💬 No function calls. Model responded with:")
        for part in candidate.parts:
            if hasattr(part, 'text') and part.text:
                print(f"   {part.text}")

# Test parallel function calling with a query requiring both weather and restaurant info
print("Testing parallel function calling:")
handle_parallel_function_calls("What's the weather like in Tokyo, Japan and can you also find some good sushi restaurants there?")

Testing parallel function calling:
📝 User prompt: What's the weather like in Tokyo, Japan and can you also find some good sushi restaurants there?
🔧 Found 2 function call(s):

  1. Function: get_current_weather
     Arguments: {'location': 'Tokyo, Japan'}
🌤️  Getting weather for Tokyo, Japan...
     Result: {'location': 'Tokyo, Japan', 'temperature': 22, 'unit': 'celsius', 'description': 'Partly cloudy', 'humidity': 65, 'wind_speed': 10}

  2. Function: search_restaurants
     Arguments: {'cuisine': 'sushi', 'location': 'Tokyo, Japan'}
🍽️  Searching restaurants in Tokyo, Japan...
     Result: {'location': 'Tokyo, Japan', 'cuisine': 'sushi', 'price_range': 'moderate', 'results': [{'name': 'Bella Vista', 'rating': 4.5, 'price': '$$', 'cuisine': 'sushi'}, {'name': 'Golden Dragon', 'rating': 4.2, 'price': '$$$', 'cuisine': 'sushi'}, {'name': 'Local Bistro', 'rating': 4.0, 'price': '$', 'cuisine': 'sushi'}]}

💬 Final natural language response:
   The weather in Tokyo, Japan is partly cloudy

Here, we define a function that takes a user prompt and sends it to Gemini, expecting that multiple function calls might be returned.
* After sending the prompt, it loops through all response parts, looking for structured function\_call objects Gemini might have generated. Each detected function call is appended to a list.
* For each function call detected, it matches the function name against known handlers, and executes the corresponding function with the extracted arguments, capturing the result alongside the function call metadata.
* Once all function calls have been handled, it constructs a structured conversation history, including the original user message, the model’s function call outputs, and the user’s function response messages.
* Finally, it sends this complete conversation history back to Gemini to generate a single, coherent natural language response incorporating all retrieved data.

#### Advanced parallel calling with conversation history
For production applications, we will want to integrate parallel function calling with persistent conversation history:

In [10]:
class ConversationManager:
    def __init__(self):
        # Initialize the Gemini client and register both tools (functions)
        self.client = genai.Client()
        self.tools = types.Tool(function_declarations=[weather_function, restaurant_function])
        self.config = types.GenerateContentConfig(tools=[self.tools])
        self.conversation_history = []

    def execute_function(self, function_call):
        """Execute a function based on its name and arguments."""
        # Dispatch to the correct function based on the name provided by Gemini
        if function_call.name == "get_current_weather":
            return get_current_weather(**dict(function_call.args))
        elif function_call.name == "search_restaurants":
            return search_restaurants(**dict(function_call.args))
        else:
            return {"error": f"Unknown function: {function_call.name}"}

    def send_message(self, message_text: str):
        """Send a message and handle potential parallel function calls."""
        # Add the user's new message to the ongoing conversation history
        self.conversation_history.append({"role": "user", "parts": [{"text": message_text}]})

        # Get model response within the context of the conversation so far
        response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=self.conversation_history,
            config=self.config,
        )

        candidate = response.candidates[0].content  # Use the top response from Gemini

        # Extract all function calls detected in the model’s response parts
        function_calls = []
        for part in candidate.parts:
            if hasattr(part, 'function_call') and part.function_call:
                function_calls.append(part.function_call)

        if function_calls:
            print(f"🔧 Executing {len(function_calls)} function(s):")

            # Execute all functions
            function_results = []  # Will store each function call with its execution result
            for function_call in function_calls:
                print(f"  • {function_call.name}({dict(function_call.args)})")
                result = self.execute_function(function_call)  # Execute and capture result
                function_results.append((function_call, result))

            # Add model's function calls to history
            self.conversation_history.append({
                "role": "model",
                "parts": [{"function_call": fc} for fc, _ in function_results]
            })

            # Add function responses to history
            function_response_parts = []
            for function_call, result in function_results:
                function_response_parts.append({
                    "function_response": {
                        "name": function_call.name,
                        "response": result
                    }
                })

            self.conversation_history.append({"role": "user", "parts": function_response_parts})

            # Ask Gemini again: now it has both the user’s original query and the function results in context
            final_response = self.client.models.generate_content(
                model="gemini-2.5-flash",
                contents=self.conversation_history,
                config=self.config,
            )

            print("💬 Model response:")
            for part in final_response.candidates[0].content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"   {part.text}")

            # Add final response to history
            self.conversation_history.append({
                "role": "model",
                "parts": final_response.candidates[0].content.parts
            })

        else:
            # If Gemini just responds directly without asking to call any functions
            print("💬 Model response (no function calls):")
            for part in candidate.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"   {part.text}")

            # Add response to history
            self.conversation_history.append({"role": "model", "parts": candidate.parts})

# Example usage of conversation manager with parallel function calling
conversation_manager = ConversationManager()

print("=== Conversation with Parallel Function Calling ===")
conversation_manager.send_message("Hello!")
conversation_manager.send_message("I'm planning a trip to Rome, Italy. Can you tell me the weather there and suggest some Italian restaurants?")
conversation_manager.send_message("Thanks! What about Paris, France? Weather and French restaurants please.")

=== Conversation with Parallel Function Calling ===
💬 Model response (no function calls):
   Hello! How can I help you today?

🔧 Executing 2 function(s):
  • get_current_weather({'location': 'Rome, Italy'})
🌤️  Getting weather for Rome, Italy...
  • search_restaurants({'cuisine': 'italian', 'location': 'Rome, Italy'})
🍽️  Searching restaurants in Rome, Italy...
💬 Model response:
   The current weather in Rome, Italy is partly cloudy with a temperature of 22 degrees Celsius. The humidity is 65% and the wind speed is 10 km/h.

Here are some Italian restaurants in Rome:

*   **Bella Vista:** Rated 4.5 stars, with a moderate price range.
*   **Local Bistro:** Rated 4 stars, with a budget-friendly price range.

Please note that "Golden Dragon" was also listed, but it does not appear to be an Italian restaurant.
🔧 Executing 2 function(s):
  • get_current_weather({'location': 'Paris, France'})
🌤️  Getting weather for Paris, France...
  • search_restaurants({'location': 'Paris, France', 'cuisi

#### Key points about parallel function calling

- Efficiency: Multiple functions can be executed simultaneously, reducing overall response time
- Natural language understanding: Gemini automatically identifies when multiple functions are needed from a single prompt
- Conversation continuity: All function calls and responses are properly integrated into the conversation history

Parallel function calling makes our AI assistant more capable of handling complex, multi-faceted queries while maintaining conversation flow and context. This is essential for building sophisticated AI applications that can efficiently interact with multiple data sources and services.